In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
from IPython.display import display, Latex, Image

import params as st
from model import *
from feedforward import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.integrate import solve_ivp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Steuerung beim Magnetlager

In [ ]:
imag = Image("../images/magnetlager.png", width=125)
display(imag)

**Modellgleichungen**

\begin{align*}
    m \ddot{y}(t) & = k \frac{i^2(t)}{(y(t)-s_0)^2} - m g
\end{align*}

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 5, 501)

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax3 = plt.subplot(222)
    ax4 = plt.subplot(224)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([tSim[0], tSim[-1]]) 
ax2.set_xlim([tSim[0], tSim[-1]]) 
ax4.set_xlim([tSim[0], tSim[-1]]) 
ax1.set_ylim([-1, 6]) 
ax2.set_ylim([0, 2]) 
ax4.set_ylim([-5, 5]) 
ax1.grid() 
ax2.grid()
ax4.grid()
ax3.set_xlim(-2, 2)
ax3.set_ylim(-1, 3)
ax3.set_xticks([])
ax3.set_yticks([])
ax1.set_ylabel(r"$u$")
ax2.set_xlabel(r"$t$")
ax2.set_ylabel(r"$y$ in m")
ax4.set_ylabel(r"$\dot{y}$ in m/s")
ax4.set_xlabel(r"$t$")

lineY, = ax2.plot([], [], label=r"ist")
lineYref, = ax2.plot([], [], '--', label=r"soll")
linedY, = ax4.plot([], [])
linedYref, = ax4.plot([], [], '--')
lineU, = ax1.plot([], [])

ballAni = ax3.add_patch(Ellipse((0, 0), st.r, st.b, facecolor='0.', edgecolor='0.'))
magAni = ax3.add_patch(plt.Rectangle((-st.r, st.r + st.s), 2 * st.r, st.h, facecolor='0.', edgecolor='0.'))

handlesAx, labelsAx = ax2.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=2, mode="expand", borderaxespad=0., framealpha=0.5)

playB = widgets.Play(value=0,
                     min=0, 
                     max=len(tSim),
                     step=10)
sliderB = widgets.IntSlider(value=0,
                            min=0,
                            max=len(tSim),
                            step=10)
sliderT0 = widgets.FloatSlider(value=0,
                               min=0,
                               max=5,
                               step=1,
                               description=r'$t_0$')
sliderT = widgets.FloatSlider(value=1,
                               min=0.5,
                               max=4,
                               step=.5,
                               description=r'$T$')
sliderYd = widgets.widgets.FloatSlider(value=0.5,
                                       min=0.2,
                                       max=0.7,
                                       step=0.1,
                                       description=r"$y_\text{d}$")
sliderY0 = widgets.widgets.FloatSlider(value=1,
                                       min=0.5,
                                       max=1.5,
                                       step=0.1,
                                       description=r"$y_\text{0}$")
radioU = widgets.RadioButtons(options=['Rampe', 'flach'],
                              description='Eingang:',
                              disabled=False)

def updateOde(_):
    global res

    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    y0 = sliderY0.value

    def sys(t, x, u, params):
        dx = nonlinSys(t, x, u, params)

        return dx

    params = st.g, st.m, st.s0 , st.k
    if radioU.value == 'Rampe':
        uS = stationarySys(y0, params)
        x0 = [y0, 0]
        uIn = feedForwardRamp(t0, T, yd, uS)
    else:
        x0 = [y0, 0]
        _, uIn = feedForwardFlat(t0, T, yd, y0)

    res = solve_ivp(sys,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim,
                    args=(uIn, params))

def updatePlot(change):
    idx = change['new']

    y = res.y.T[idx, 0]
    dy = res.y.T[idx, 1]
    
    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    y0 = sliderY0.value

    params = st.g, st.m, st.s0 , st.k
    if radioU.value == 'Rampe':
        uS = stationarySys(y0, params)
        uIn = feedForwardRamp(t0, T, yd, uS)
        lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
        lineY.set_data(tSim[:idx], res.y.T[:idx, 0])
        lineYref.set_data([], [])
        linedY.set_data(tSim[:idx], res.y.T[:idx, 1])
        linedYref.set_data([], [])
    else:
        yr, uIn = feedForwardFlat(t0, T, yd, y0)
        lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
        lineY.set_data(tSim[:idx], res.y.T[:idx, 0])
        lineYref.set_data(tSim[:idx], [yr[0](_t) for _t in tSim[:idx]])
        linedY.set_data(tSim[:idx], res.y.T[:idx, 1])
        linedYref.set_data(tSim[:idx], [yr[1](_t) for _t in tSim[:idx]])

    ballAni.set_center((0, y - 1))
    
    fig.canvas.draw()    

sliderB.observe(updatePlot, names='value')
sliderT0.observe(updateOde, names='value')
sliderT.observe(updateOde, names='value')
sliderYd.observe(updateOde, names='value')
sliderY0.observe(updateOde, names='value')
radioU.observe(updateOde, names='value')

updateOde(_)

ffControls = VBox([sliderY0, sliderYd, sliderT0, sliderT])
ffControls.layout = make_box_layout()
inputControls = Box([radioU])
inputControls.layout = make_box_layout()
controls = HBox([ffControls, inputControls])

jslink((playB, 'value'), (sliderB, 'value'))
videoControls = VBox([HBox([playB, sliderB]), output])
videoControls.layout = make_box_layout()

HBox([controls, Box([videoControls])], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))